In [ ]:
'''
import sys
sys.path.append('/Users/jiangxiaoyu/Desktop/All Projects/Scalable_LVMOGP/')
from code_blocks.kernels.periodic_inputs_maternkernel import PeriodicInputsMaternKernel
from code_blocks.likelihoods.gaussian_likelihood import GaussianLikelihood
from gpytorch.kernels import MaternKernel
import torch

original_kernel = MaternKernel(nu=2.5)
test_kernel = PeriodicInputsMaternKernel(nu=2.5, period=0.33)
test_likelihood = GaussianLikelihood()
'''

In [ ]:
'''
test_tensor = torch.tensor([0.5, 1, 2, 3])
test_kernel(test_tensor).to_dense()
x1 = 2 * test_tensor * torch.pi / 0.33
x1 = torch.cat((torch.sin(x1).reshape(-1, 1), torch.cos(x1).reshape(-1, 1)), axis=1)
original_kernel(x1).to_dense()
'''

In [ ]:
'''
original_kernel.lengthscale = 15.
for name, params in original_kernel.named_parameters():
    print(name)
    print(params)
'''

In [ ]:
'''
import yaml

root_config = '/Users/jiangxiaoyu/Desktop/All Projects/Scalable_LVMOGP/configs/' 
# NOTE: Specify name here for different experiments: 
# rnd (fix) + unfix (fix) ; first referring to initialization, second referring to inducing points in input space 
curr_config_name = 'spatiotemp/Scale_Matern52_plus_Scale_PeriodicInputsMatern52/lvmogp_catlatent_rnd_unfix' 
curr_config = f'{root_config}/{curr_config_name}.yaml'
with open(curr_config, 'r') as file: 
    config = yaml.safe_load(file) 

import sys
sys.path.append('/Users/jiangxiaoyu/Desktop/All Projects/Scalable_LVMOGP/')
from code_blocks.likelihoods.gaussian_likelihood import GaussianLikelihood
from gplvm_init import GPLVM, train_gplvm
from run_experiments.prepare_dataset import prepare_spatio_temp_data
import torch


_, data_Y_squeezed, *arg = prepare_spatio_temp_data(config=config)

data_Y = data_Y_squeezed.reshape(config['n_outputs'], config['n_input'])[:, :config['n_input_train']]

class GaussianLikelihoodWithMissingObs(GaussianLikelihood):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    @staticmethod
    def _get_masked_obs(x):
        missing_idx = x.isnan()
        x_masked = x.masked_fill(missing_idx, -999.)
        return missing_idx, x_masked

    def expected_log_prob(self, target, input, *params, **kwargs):
        missing_idx, target = self._get_masked_obs(target)
        res = super().expected_log_prob(target, input, *params, **kwargs)
        return res * ~missing_idx

    def log_marginal(self, observations, function_dist, *params, **kwargs):
        missing_idx, observations = self._get_masked_obs(observations)
        res = super().log_marginal(observations, function_dist, *params, **kwargs)
        return res * ~missing_idx
    
my_GPLVM = GPLVM(n = config['n_outputs'], 
                 data_dim= config['n_input_train'], 
                 latent_dim = 2, 
                 n_inducing = 20)

my_likelihood = GaussianLikelihoodWithMissingObs()

_, _, losses = train_gplvm(my_GPLVM, my_likelihood, data_Y)

import matplotlib.pyplot as plt
plt.plot(losses)
'''

# Test cached function

In [ ]:
# from gpytorch.utils.memoize import cached

In [ ]:
'''
class class_test():

    def __init__(self, a=1.):
        self.a = a
    
    # @property
    @cached(name='value_b')
    def value_b(self):
        return self.a ** 2

class_test = class_test(a=10.)
print(class_test.value_b())
print(class_test._memoize_cache)
class_test.a = 11.
print(class_test.a)
print(class_test.value_b())
print(class_test._memoize_cache)

class class_test_2():

    def __init__(self, a=1.):
        self.a = a
    
    # @property
    @cached(name='value_b')
    def value_b(self, c):
        return self.a ** c
    
class_test_2 = class_test_2(a=10.)
print(class_test_2.value_b(c=2.))
print(class_test_2._memoize_cache)

class_test_2.a = 11.

print(class_test_2.value_b(c=2.))
print(class_test_2._memoize_cache)

print(class_test_2.value_b(c=3.))
print(class_test_2._memoize_cache)

import pickle

pickle.dumps({'c': 2.})
'''

# Test KL cmputation

In [ ]:
import torch 
from torch import Tensor
from linear_operator.operators import CholLinearOperator, TriangularLinearOperator, KroneckerProductLinearOperator, DiagLinearOperator
import time
from gpytorch.distributions import MultivariateNormal
from torch.distributions import MultivariateNormal as TMultivariateNormal

In [ ]:
torch.set_default_dtype(torch.double)

In [ ]:
def kl_divergence_kronecker_wrt_identity_1(chol_variational_covar_latent: Tensor, 
                                           chol_variational_covar_input: Tensor, 
                                           variational_mean: Tensor):
    
    M_X, M_H = chol_variational_covar_input.shape[0], chol_variational_covar_latent.shape[0]
    _variational_mean = variational_mean.reshape(M_H, M_X)
    # log_det_varcov_latent = (torch.prod(torch.diag(chol_variational_covar_latent)) ** 2).log()
    # log_det_varcov_input = (torch.prod(torch.diag(chol_variational_covar_input)) ** 2).log()
    log_det_varcov_latent = 2 * torch.sum(torch.log(torch.diag(chol_variational_covar_latent)))
    log_det_varcov_input = 2 * torch.sum(torch.log(torch.diag(chol_variational_covar_input)))
    tr_varcov_latent = torch.sum(chol_variational_covar_latent ** 2)
    tr_carcov_input = torch.sum(chol_variational_covar_input ** 2)
    tr_MTM = torch.norm(_variational_mean, p='fro') ** 2 # trace(M^TM) = F_norm(M)^2
    
    res = - M_X * log_det_varcov_latent - M_H * log_det_varcov_input + tr_MTM + tr_carcov_input*tr_varcov_latent - M_X*M_H 
    return res/2



def kl_divergence_kronecker_wrt_identity_2(chol_variational_covar_latent: Tensor, 
                                           chol_variational_covar_input: Tensor, 
                                           variational_mean: Tensor):
    '''KL divergence of q and p, where q has kronecker product covariance and p is standard normal distribution.'''
    
    M_X, M_H = chol_variational_covar_input.shape[0], chol_variational_covar_latent.shape[0]
    _variational_mean = variational_mean.reshape(M_H, M_X)
    log_det_varcov_latent = (torch.prod(torch.diag(chol_variational_covar_latent)) ** 2).log()
    log_det_varcov_input = (torch.prod(torch.diag(chol_variational_covar_input)) ** 2).log()
    tr_varcov_latent = torch.sum(chol_variational_covar_latent ** 2)
    tr_carcov_input = torch.sum(chol_variational_covar_input ** 2)
    tr_MTM = torch.norm(_variational_mean, p='fro') ** 2 # trace(M^TM) = F_norm(M)^2
    
    res = - M_X * log_det_varcov_latent - M_H * log_det_varcov_input + tr_MTM + tr_carcov_input*tr_varcov_latent - M_X*M_H 
    return res/2

In [ ]:
M_H = 40
M_X = 50
L_H_1 = torch.randn(M_H, M_H)
L_X_1 = torch.randn(M_X, M_X)
# _L_H = torch.eye(M_H).to(torch.double)
# _L_X = torch.eye(M_X).to(torch.double)
variational_mean = torch.randn(M_X * M_H)
# variational_mean[:10] = torch.pi

lower_mask_latent = torch.ones(L_H_1.shape[-2:]).tril(0)
lower_mask_input = torch.ones(L_X_1.shape[-2:]).tril(0)
L_H = L_H_1.mul(lower_mask_latent)
L_X = L_X_1.mul(lower_mask_input)

for i in range(L_H.size(0)): 
    L_H[i, i] = L_H[i, i] ** 2 + 1.


for i in range(L_X.size(0)): 
    L_X[i, i] = L_X[i, i] ** 2 + 1.

In [ ]:
# varcov = KroneckerProductLinearOperator(CholLinearOperator(TriangularLinearOperator(L_H)), CholLinearOperator(TriangularLinearOperator(L_X))).to_dense()
# varcov_ = torch.kron(CholLinearOperator(TriangularLinearOperator(L_H)).to_dense(), CholLinearOperator(TriangularLinearOperator(L_X)).to_dense())

# (varcov_ - varcov).max()

In [ ]:
varcov = KroneckerProductLinearOperator(CholLinearOperator(TriangularLinearOperator(L_H)), CholLinearOperator(TriangularLinearOperator(L_X)))
# varcov_ = torch.kron(CholLinearOperator(TriangularLinearOperator(L_H)).to_dense(), CholLinearOperator(TriangularLinearOperator(L_X)).to_dense())
q_u = MultivariateNormal(variational_mean, varcov)
zeros = torch.zeros(variational_mean.shape[0])
p_u = MultivariateNormal(zeros, DiagLinearOperator(torch.ones_like(zeros)))

q_u_2 = TMultivariateNormal(variational_mean, varcov.to_dense())
p_u_2 = TMultivariateNormal(zeros, torch.eye(zeros.shape[0]))

# make sure diag all positive ... 
start_time = time.time()
L_H_ = torch.linalg.cholesky(CholLinearOperator(TriangularLinearOperator(L_H)).to_dense())
L_X_ = torch.linalg.cholesky(CholLinearOperator(TriangularLinearOperator(L_X)).to_dense())
end_time = time.time()

print('total time', end_time - start_time)

In [ ]:
torch.all(q_u.covariance_matrix == q_u_2.covariance_matrix)

#### 2 questions:
1. why varcov.to_dense() not psd? It should be. Especially when matrix goes large ... 
2. why Gpytorch based MultivariateNormal havn't report this issue?

In [ ]:
'''
_variational_mean = variational_mean.reshape(M_H, M_X)
true_KL = 0.5 * (torch.diag(_variational_mean @ _variational_mean.T).sum())
print('true_KL', true_KL)
'''

In [ ]:
start_time = time.time()
res_1 = kl_divergence_kronecker_wrt_identity_2(L_H_, L_X_, variational_mean)
end_time = time.time()
print('Our method log sum', res_1)
print('total time:', end_time - start_time)

In [ ]:
start_time = time.time()
res_1 = kl_divergence_kronecker_wrt_identity_1(L_H_, L_X_, variational_mean)
end_time = time.time()
print('Our method prod log', res_1)
print('total time:', end_time - start_time)

In [ ]:
start_time = time.time()
res_2 = torch.distributions.kl.kl_divergence(q_u, p_u)
end_time = time.time()
print('GpyTorch method', res_2)
print('total time:', end_time - start_time)

In [ ]:
start_time = time.time()
res_2 = torch.distributions.kl.kl_divergence(q_u_2, p_u_2)
end_time = time.time()
print('torch method', res_2)
print('total time:', end_time - start_time)

In [ ]:
'''
def _batch_trace_XXT(bmat):
    """
    Utility function for calculating the trace of XX^{T} with X having arbitrary trailing batch dimensions
    """
    n = bmat.size(-1)
    m = bmat.size(-2)
    flat_trace = bmat.reshape(-1, m * n).pow(2).sum(-1)
    return flat_trace.reshape(bmat.shape[:-2])

from torch.distributions.multivariate_normal import _batch_mahalanobis

def _kl_multivariatenormal_multivariatenormal(p, q):
    # From https://en.wikipedia.org/wiki/Multivariate_normal_distribution#Kullback%E2%80%93Leibler_divergence
    if p.event_shape != q.event_shape:
        raise ValueError("KL-divergence between two Multivariate Normals with\
                          different event shapes cannot be computed")

    # why p._unbroadcasted_scale_tril returns negative values ?
    half_term1 = (q._unbroadcasted_scale_tril.diagonal(dim1=-2, dim2=-1).log().sum(-1) -
                  torch.linalg.cholesky(q_u.covariance_matrix).diag().log().sum(-1))
    
    combined_batch_shape = torch._C._infer_size(q._unbroadcasted_scale_tril.shape[:-2],
                                                p._unbroadcasted_scale_tril.shape[:-2])
    n = p.event_shape[0]
    q_scale_tril = q._unbroadcasted_scale_tril.expand(combined_batch_shape + (n, n))
    p_scale_tril = torch.linalg.cholesky(q_u.covariance_matrix).expand(combined_batch_shape + (n, n))
    term2 = _batch_trace_XXT(torch.linalg.solve_triangular(q_scale_tril, p_scale_tril, upper=False))
    term3 = _batch_mahalanobis(q._unbroadcasted_scale_tril, (q.loc - p.loc))
    return half_term1 + 0.5 * (term2 + term3 - n)

start_time = time.time()
res_2 = _kl_multivariatenormal_multivariatenormal(q_u, p_u)
end_time = time.time()
print(res_2)
print('total time:', end_time - start_time)
'''

# Test KL between latent variables

In [ ]:
import torch

In [ ]:
q_mu = torch.zeros(100, 2)
q_sigma = torch.ones(100, 2) + 1.

batch_idx = [i for i in range(10)]

q_mu_batch = q_mu[batch_idx, ...]
q_sigma_batch = q_sigma[batch_idx, ...]

# --- --- --- ---
p_mu = torch.zeros(100, 2)
p_sigma = torch.ones(100, 2)

p_mu_batch = p_mu[batch_idx, ...]
p_sigma_batch = p_sigma[batch_idx, ...]

In [ ]:
q_x = torch.distributions.Normal(q_mu_batch, q_sigma_batch)
p_x = torch.distributions.Normal(p_mu_batch, p_sigma_batch)

In [ ]:
torch.distributions.kl_divergence(q_x, p_x)

# Test elementwise product with linear operator

In [1]:
import torch
from linear_operator.operators import KroneckerProductLinearOperator, TriangularLinearOperator, LinearOperator, CholLinearOperator, DenseLinearOperator, MatmulLinearOperator, SumLinearOperator

In [5]:
A = torch.eye(4) * torch.tensor([1, 2, 4, 5])
B = torch.eye(4) * torch.rand(4)
C = torch.tensor([[10., 0.], [0., 30.]])
D = torch.tensor([[15., 0.], [0., 70.]])
E = torch.randn(4, 4)

In [6]:
AmulB_lt = SumLinearOperator(A, B)
C_lt = KroneckerProductLinearOperator(C, D)
E_lt = DenseLinearOperator(E)

In [7]:
AmulB_lt.mul(C_lt).to_dense()

tensor([[  205.1507,     0.0000,     0.0000,     0.0000],
        [    0.0000,  1533.7067,     0.0000,     0.0000],
        [    0.0000,     0.0000,  1840.7195,     0.0000],
        [    0.0000,     0.0000,     0.0000, 11895.7988]])

In [8]:
( A + B ).mul(torch.kron(C, D))

tensor([[  205.1507,     0.0000,     0.0000,     0.0000],
        [    0.0000,  1533.7070,     0.0000,     0.0000],
        [    0.0000,     0.0000,  1840.7194,     0.0000],
        [    0.0000,     0.0000,     0.0000, 11895.7988]])

In [ ]:
( A + (B) ) * torch.kron(C, D)